<a href="https://colab.research.google.com/github/yangchangnaihoby/AI_7th/blob/master/Project_004_CV_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytesseract
!pip install imutils
!pip install selenium

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import re
import requests
import pytesseract
import imutils
from imutils.perspective import four_point_transform
from PIL import ImageFont, ImageDraw, Image
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
img = cv2.imread('cvimage/sample_2.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

font = ImageFont.truetype('fonts/gulim.ttc', 20)

# img_h, img_w = img.shape[ : 2]
# boxes = pytesseract.image_to_boxes(img)
# for b in boxes.splitlines() :
#     b = b.split(' ')
#     x, y, w, h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
#     cv2.rectangle(img, (x, img_h - y), (w, img_h - h), (0, 0, 255), 1)
#     cv2.putText(img, b[0], (x, img_h - y + 20), cv2.FONT_HERSHEY_COMPLEX, 1, (50, 50, 255), 1)

img_h, img_w = img.shape[ : 2]
boxes = pytesseract.image_to_data(img, lang = 'kor+eng')
for count, b in enumerate(boxes.splitlines()) :
    if count != 0 :
        b = b.split()
        if len(b) == 12 :
            x, y, w, h = int(b[6]), int(b[7]), int(b[8]), int(b[9])
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 1)
#             cv2.putText(img, b[11], (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, (50, 50, 255), 1)
            img = Image.fromarray(img)
            draw = ImageDraw.Draw(img)
            draw.text((x, y - 20), b[11], font = font, fill = (50, 50, 255))
            img = np.array(img)
            print(b[11], end = ' ')

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS) * 0.8)
fourcc = cv2.VideoWriter_fourcc(*'DIVX')

out = cv2.VideoWriter('review.out', fourcc, fps, (w, h))

while True :
    
    ret, frame = cap.read()
    
    if not ret :
        print('No video alert!!')
        break
        
    edge = cv2.Canny(frame, 50, 150)
    
    out.write(frame)
    cv2.imshow('frame', frame)
    cv2.imshow('edge', edge)
    
    if cv2.waitKey(30) == 27 :
        break
        
out.release()
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)

font = ImageFont.truetype('fonts/gulim.ttc', 20)

cap_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
cap_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS) * 0.8)
fourcc = cv2.VideoWriter_fourcc(*'DIVX')

out = cv2.VideoWriter('review.out', fourcc, fps, (cap_w, cap_h))

word_dict = {}

while True :
    
    ret, frame = cap.read()
    
    if not ret :
        print('No video alert!!')
        break
        
    frame = cv2.medianBlur(frame, ksize = 3)
    boxes = pytesseract.image_to_data(frame, lang = 'kor+eng')
    
    for count, b in enumerate(boxes.splitlines()) :
        
        if count != 0 :
            b = b.split()
            
            if len(b) == 12 :
                x, y, w, h = int(b[6]), int(b[7]), int(b[8]), int(b[9])
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 1)
                frame = Image.fromarray(frame)
                draw = ImageDraw.Draw(frame)
                draw.text((x, y - 20), b[11], font = font, fill = (50, 50, 255))
                frame = np.array(frame)
                b_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', b[11])
                b_rmv = b_rmv.strip()
                
                if b_rmv not in word_dict :
                    word_dict[b_rmv] = 1
                else :
                    word_dict[b_rmv] += 1
                                        
    out.write(frame)
    cv2.imshow('Recognizing', frame)
    word_list = sorted(word_dict.items(), key = lambda x : x[1], reverse = True)
    
    if (cv2.waitKey(30) == 27) :
        break

    if word_list :
        if word_list[0][0] != '' :
            if word_list[1][1] >= 10 :
                break
        else :
            if word_list[0][1] >= 10 :
                break
        
out.release()
cap.release()
cv2.destroyAllWindows()

sentence_to_search = ''            
            
for i in range(len(word_list)) :
    if (word_list[i][1] >= 5) and (word_list[i][0] != '') :
        sentence_to_search += word_list[i][0]
        sentence_to_search += ' '

sentence_to_search.rstrip()
print(sentence_to_search)

user_agent = 'Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36'

firefox_options = Options()
firefox_options.add_argument('user-agent=' + user_agent)
driver = webdriver.Firefox(executable_path = r'C:/Users/USER/Desktop/python_class/geckodriver.exe')

path = 'https://www.google.com/search?q=' + sentence_to_search + '&client=firefox-b-d&sxsrf=ALiCzsbmzWVPASKAPg9YZDRIKYIAtoCkYg:1658816286149&source=lnms&tbm=isch&sa=X&ved=2ahUKEwj33e6b9JX5AhUYplYBHeIGBYQQ_AUoAXoECAEQAw&biw=852&bih=941&dpr=1.5'

driver.get(path)

# 교수님 comment : GaussianBlur, 3 frames 당 평균으로

Ag Home Test COVID 19 


C:\Users\USER\AppData\Local\Temp\ipykernel_7016\714778888.py:78: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path = r'C:/Users/USER/Desktop/python_class/geckodriver.exe')
